# Open Source Production Cost Modeling in Julia

**Dheepak Krishnamurthy**

National Renewable Energy Laboratory (NREL), Golden, CO

*July 23rd*

JuliaCon 2019, Baltimore, MD

[http://github.com/kdheepak/juliacon2019-open-source-power-system-production-cost-modeling-in-julia](http://github.com/kdheepak/juliacon2019-open-source-power-system-production-cost-modeling-in-julia)

Launch a live notebook and follow along:

[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/kdheepak/juliacon2019-open-source-power-system-production-cost-modeling-in-julia/master)


In [1]:
# initialization

In [2]:
] activate .

In [3]:
using Suppressor;

@suppress begin
    using PowerSystems;
    using PowerSimulations;
    using JuMP
    using TimeSeries;
    using GLPK; 
    using DataFrames;
    using D3TypeTrees;
end

# Table of Contents

- What is Production Cost Modeling?
- Modeling Demonstration
- PowerSystems.jl and PowerSimulations.jl
- Learnings and Conclusions

# What is Production Cost Modeling?

## Electricity Grid Operations

- Supply of electricity must meet Demand at all times
- Generation of electricity (Supply) is not always co-located with Load (Demand)
- Electricity flows through large interconnected network / grid
- Generation located at different points on the grid may have different costs


## Production Cost Modeling

- Captures all the costs of operating a fleet of generators that are connected to a given electrical grid 
- Includes Economic Dispatch, Unit Commitment, and Location Marginal Prices (LMPs)


## Why Production Cost Modeling?

- Understand how efficiently the grid is being operated
- Evaluate the economic impacts of decisions
- Verify feasibility of planning outcomes

## Mathematical Programming for Production Cost Modeling

- Can be formulated as an optimization problem
- Mixed Integer Linear Problem (MILP)
- [JuMP.jl](https://github.com/JuliaOpt/JuMP.jl) is a domain-specific modeling language for mathematical optimization embedded in [Julia](https://github.com/JuliaLang/julia/). 

# Modeling demonstration

<img src="./images/5busT.png" style="width: 50%;"/>

<!-- 
generators5 = vcat(sys.generators.thermal[:], sys.generators.renewable[:])
-->

In [4]:
include("./scripts/read_data.jl")
getproperty.(generators5, :name)[1:5]

5-element Array{String,1}:
 "Alta"     
 "Park City"
 "Solitude" 
 "Sundance" 
 "Brighton" 

In [5]:
sys;

In [6]:
@assert sys isa PowerSystems.System

In [7]:
@assert generators5 isa Vector{PowerSystems.Generator}

In [8]:
@assert (generators5[1] |> typeof) === PowerSystems.ThermalDispatch

In [9]:
economic_dispatch_model = JuMP.Model()

A JuMP Model
Feasibility problem with:
Variables: 0
Model mode: AUTOMATIC
CachingOptimizer state: NO_OPTIMIZER
Solver name: No optimizer attached.

$$
\begin{aligned}
& p_{g,t} >= P^{MIN}_g & & \forall g,t \\
& p_{g,t} <= P^{MAX}_g & & \forall g,t \\
\end{aligned}
$$

- $p_{g,t}$ is the MW produced by generator $g$ in period $t$,
- $P_g^{MAX}$ and $P_g^{MIN}$ are the maximum and minimum MW respectively that can be produced by generator $g$

In [10]:
generators5[1]

ThermalDispatch:
   name: Alta
   available: true
   bus: Bus(name="nodeA")
   tech: TechThermal
   econ: EconThermal

In [11]:

g_set = [generators5[i].name for i in 1:5]
@variable(economic_dispatch_model, Pg[g_set, t = 1:1])

for i = 1:5
    @constraint(economic_dispatch_model, Pg[generators5[i].name, 1] >= generators5[i].tech.activepowerlimits.min)
    @constraint(economic_dispatch_model, Pg[generators5[i].name, 1] <= generators5[i].tech.activepowerlimits.max)
end

In [12]:
economic_dispatch_model

A JuMP Model
Feasibility problem with:
Variables: 5
`GenericAffExpr{Float64,VariableRef}`-in-`MathOptInterface.GreaterThan{Float64}`: 5 constraints
`GenericAffExpr{Float64,VariableRef}`-in-`MathOptInterface.LessThan{Float64}`: 5 constraints
Model mode: AUTOMATIC
CachingOptimizer state: NO_OPTIMIZER
Solver name: No optimizer attached.
Names registered in the model: Pg

- Different kinds of generators
    - Thermal
    - Renewable
    - Hydro

In [13]:
supertype.(typeof.(generators5))

7-element Array{DataType,1}:
 ThermalGen  
 ThermalGen  
 ThermalGen  
 ThermalGen  
 ThermalGen  
 RenewableGen
 RenewableGen

In [14]:
function powerconstraints(m, P_g, generator::ThermalGen)
    for var in P_g
        @constraint(m, var >= generator.tech.activepowerlimits.min)
        @constraint(m, var <= generator.tech.activepowerlimits.max)
    end
end


function powerconstraints(m, P_g, generator::RenewableGen)
    for (time, var) in enumerate(P_g)
        @constraint(m, var <= generator.tech.installedcapacity*values(generator.scalingfactor)[time])
    end
end



powerconstraints (generic function with 2 methods)

In [15]:
economic_dispatch_model = JuMP.Model()
@variable(economic_dispatch_model, Pg[g_set, t = 1:sys.time_periods] >=0 );
for (ix, name) in enumerate(Pg.axes[1])
    @assert name == generators5[ix].name
    powerconstraints(economic_dispatch_model, Pg[name, :], generators5[ix])
end

<!--Start equation--> $$
\begin{aligned}
& \sum_{g}^{G} p_{g,t} = \sum_{d}^{D} p_{d,t} & & \forall t \\
\end{aligned}
$$<!--End equation-->

- $p_{g,t}$ is the MW produced by generator $g$ in period $t$,
- $p_{d,t}$ is the MW consumed by load $d$ in period $t$,


In [16]:

for time in 1:sys.time_periods
    @constraint(economic_dispatch_model, sum(Pg[g, time] for g in g_set) == sum(loads5_DA[l].maxactivepower*values(loads5_DA[l].scalingfactor) for l in 1:4)[time])
end

$$
\begin{aligned}
\underset{p_{g,t}}{\text{min}}
\sum_{t}^{T}\sum_{g}^{G} p_{g,t} PC_{g,t}
\end{aligned}
$$

- $p_{g,t}$ is the MW produced by generator $g$ in period $t$,
- $PC_{g,t}$ is production cost (\\$/MW/period) of operating generator $g$ in period $t$,


In [17]:
generators5[1]

ThermalDispatch:
   name: Alta
   available: true
   bus: Bus(name="nodeA")
   tech: TechThermal
   econ: EconThermal

In [18]:
c = []

for t in 1:1 # sys.time_periods

    #the first four generators have a variable cost defined as a julia function
    for g in 1:4
        push!(c, generators5[g].econ.variablecost(Pg[g_set[g], t]))
    end
    # The fifth generator has a piecewise linear variable cost, so we can simplify to a scalar function
    push!(c, Pg[g_set[5], t]*generators5[5].econ.variablecost[end][2]/generators5[5].econ.variablecost[end][1])

end

@objective(economic_dispatch_model, Min, sum(c))

14 Pg[Alta,1] + 15 Pg[Park City,1] + 30 Pg[Solitude,1] + 40 Pg[Sundance,1] + 60 Pg[Brighton,1]

In [19]:
economic_dispatch_model

A JuMP Model
Minimization problem with:
Variables: 120
Objective function type: GenericAffExpr{Float64,VariableRef}
`VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 120 constraints
`GenericAffExpr{Float64,VariableRef}`-in-`MathOptInterface.EqualTo{Float64}`: 24 constraints
`GenericAffExpr{Float64,VariableRef}`-in-`MathOptInterface.GreaterThan{Float64}`: 120 constraints
`GenericAffExpr{Float64,VariableRef}`-in-`MathOptInterface.LessThan{Float64}`: 120 constraints
Model mode: AUTOMATIC
CachingOptimizer state: NO_OPTIMIZER
Solver name: No optimizer attached.
Names registered in the model: Pg

In [20]:
optimize!(economic_dispatch_model, with_optimizer(GLPK.Optimizer))

In [21]:
JuMP.primal_status(economic_dispatch_model)

FEASIBLE_POINT::ResultStatusCode = 1

In [22]:
JuMP.value(Pg["Alta", 1])

40.0

$$
\begin{aligned}
\underset{p_{g,t}, uc_{g,t}}{\text{min}}
\ \sum_{t}^{T}\sum_{g}^{G} uc_{g,t} FC_{g,t}
+ \sum_{t}^{T}\sum_{g}^{G} p_{g,t} PC_{g,t} \\
+ \sum_{t}^{T}\sum_{g}^{G} su_{g,t} SU_{g,t}
+ \sum_{t}^{T}\sum_{g}^{G} sd_{g,t} SD_{g,t}
\end{aligned}
$$

- $p_{g,t}$ is the MW produced by generator $g$ in period $t$,
- $uc_{g,t}$ is 1 (or 0) if generator $g$ is dispatched (or not) during $t$,
- $su_{g,t}$ is 1 (or 0) if generator $g$ is started up (or not) at beginning of period $t$,
- $sd_{g,t}$ is 1 (or 0) if generator $g$ is shut down (or not) at beginning of period $t$,

- $FC_{g,t}$ is no-load cost (\\$/period) of operating generator $g$ in period $t$,
- $PC_{g,t}$ is production cost (\\$/MW/period) of operating generator $g$ in period $t$,
- $SU_{g,t}$ is startup cost (\\$) of starting generator $g$ in period $t$,
- $SD{g,t}$ is shutdown cost (\\$) of shutting generator $g$ in period $t$.


$$
\begin{aligned}
& p_{g,t} >= uc_{g,t} \times P^{MIN}_g & & \forall g,t \\
& p_{g,t} <= uc_{g,t} \times P^{MAX}_g & & \forall g,t \\
& p_{g,t} <= p_{g,t-1} + P^{RULim}_g & & \forall g,t \\
& p_{g,t} >= p_{g,t-1} - P^{RDLim}_g & & \forall g,t \\
& uc_{g,t} <= uc_{g,t-1} + su_{g,t} & & \forall g,t \\
& uc_{g,t} >= uc_{g,t-1} - sd_{g,t} & & \forall g,t \\
& \sum_{b}^{B} A[l,b] \times (\sum_g^G p_{g,t}^{b} - \sum_d^D p_{d,t}^{b}) <= MaxFlow_l & & \forall l,t \\
\end{aligned}
$$

- $p_{d,t}$ is the MW consumed by load $d$ in period $t$,
- $A[l,b]$ is the contribution factor from an injection at bus $b$ to line $l$,
- $P_g^{MAX}$ and $P_g^{MIN}$ are the maximum and minimum MW respectively that can be produced by generator $g$, $P_g^{RULim}$ and $P_g^{RDLim}$ are the MW/period that can be ramped up and ramped down respectively by generator $g$,
- $MaxFlow_l$ is the maximum flow limit on line $l$.



## Challenges with Production Cost Modeling

- Require large amount of data
    - Generator characteristics, load forecast data at various intervals, grid models
- Benchmarking and validation requires standard vetted implementations
- Significant time required to setup data, build models, simulate and compute results

# Production Cost Modeling in Julia

- [https://github.com/NREL/PowerSystems.jl](https://github.com/NREL/PowerSystems.jl)
- [https://github.com/NREL/PowerSimulations.jl](https://github.com/NREL/PowerSimulations.jl)

## PowerSystems.jl

- Rigorous data model using Julia structures to enable power systems analysis
    - Generators (Thermal, Renewable, Synchronous Condensers, and Hydro)
    - Transmission (Lines, and Transformers)
    - Active Flow control devices (DC Lines and phase-shifters)
    - Topological elements (Buses, Areas)
    - Storage (Batteries)
    - Load (Static, and curtailable)
    - Services (Reserves, inter-regional transfers)
    - Forecasts (Deterministic, scenario, stochastic)


In [23]:
TypeTree(Generator, init_expand=1)

Generator
├──RenewableGen
│  ├──RenewableFix
│  ├──RenewableCurtailment
│  └──RenewableFullDispatch
├──HydroGen
│  ├──HydroFix
│  ├──HydroCurtailment
│  └──HydroStorage
└──ThermalGen
   ├──ThermalDispatch
   └──ThermalGenSeason

In [24]:
TypeTree(PowerSystems.Component, init_expand=1)

Component
├──Service
│  ├──Reserve
│  │  ├──StaticReserve (0 children)
│  │  └──ProportionalReserve (0 children)
│  └──Transfer
├──PowerSystems.TechnicalParams
│  ├──TechRenewable
│  ├──EconHydro
│  ├──TechThermal
│  ├──TechHydro
│  ├──EconRenewable
│  └──EconThermal
├──GenClasses
├──Forecast
│  ├──Scenarios
│  ├──Probabilistic
│  └──Deterministic
└──Device
   ├──Branch
   │  ├──TapTransformer (0 children)
   │  ├──HVDCLine (0 children)
   │  ├──DCLine (1 children)
   │  ├──PhaseShiftingTransformer (0 children)
   │  ├──Transformer2W (0 children)
   │  ├──Line (0 children)
   │  └──MonitoredLine (0 children)
   └──Injection
      ├──Storage (1 children)
      ├──LoadZones (0 children)
      ├──Bus (0 children)
      ├──Generator (3 children)
      └──ElectricLoad (3 children)

## PowerSimulations.jl

- Provide a flexible modeling framework that can accommodate problems of different complexity and at different time-scales.
    - Production Cost Modeling
    - Capacity Expansion Modeling
    - Load Flow and Contingency Analysis
- Streamline the construction of large scale optimization problems to avoid repetition of work when adding/modifying model details.
- Exploit Julia's capabilities to improve computational performance of large scale power system simulations.


In [25]:
ed_model = PowerSimulations.EconomicDispatch(sys, PowerSimulations.CopperPlatePowerModel, 
                                        optimizer = with_optimizer(GLPK.Optimizer));
res_5 = @suppress solve_op_model!(ed_model);
res_5.variables[:Pth][:Alta][1]

40.0

[PowerSimulations.jl](https://github.com/nrel/powersimulations.jl) provides access to optimal scheduling formulations for devices in a `PowerSystem`. A number of different formulations are enabled by the `AbstractDeviceFormulation` type tree:

In [26]:
TypeTree(PowerSimulations.AbstractDeviceFormulation,scopesep="\n")

PowerSimulations
AbstractDeviceFormulation
├──PowerSimulations
│  AbstractThermalFormulation
│  ├──PowerSimulations
│  │  ThermalUnitCommitment
│  └──PowerSimulations
│     AbstractThermalDispatchForm
│     ├──PowerSimulations
│     │  ThermalDispatchNoMin (0 children)
│     ├──PowerSimulations
│     │  ThermalRampLimited (0 children)
│     └──PowerSimulations
│        ThermalDispatch (0 children)
├──PowerSimulations
│  AbstractBranchFormulation
│  ├──PowerSimulations
│  │  AbstractTransformerForm
│  ├──PowerSimulations
│  │  AbstractDCLineForm
│  │  └──PowerSimulations
│  │     SimpleHVDC (0 children)
│  └──PowerSimulations
│     AbstractLineForm
│     ├──PowerSimulations
│     │  PiLine (0 children)
│     └──PowerSimulations
│        SeriesLine (0 children)
├──PowerSimulations
│  AbstractLoadFormulation
│  ├──PowerSimulations
│  │  AbstractControllablePowerLoadForm
│  │  └──PowerSimulations
│  │     InterruptiblePowerLoad (0 children)
│  └──PowerSimulations
│     StaticPowerLoad
├──PowerSimulations
│  AbstractStorageForm
│  └──PowerSimulations
│     BookKeepingModel
├──PowerSimulations
│  AbstractRenewableFormulation
│  ├──PowerSimulations
│  │  AbstractRenewableDispatchForm
│  │  ├──PowerSimulations
│  │  │  RenewableFullDispatch (0 children)
│  │  └──PowerSimulations
│  │     RenewableConstantPowerFactor (0 children)
│  └──PowerSimulations
│     RenewableFixed
└──PowerSimulations
   AbstractHydroFormulation
   ├──PowerSimulations
   │  AbstractHydroDispatchForm
   │  ├──PowerSimulations
   │  │  HydroDispatchRunOfRiver (0 children)
   │  └──PowerSimulations
   │     HydroDispatchSeasonalFlow (0 children)
   ├──PowerSimulations
   │  HydroCommitmentSeasonalFlow
   └──PowerSimulations
      HydroCommitmentRunOfRiver

In addition to the formulations enabled under the `AbstractDeviceFormulation`, a deep integration with [PowerModels.jl](https://github.com/lanl-ansi/powermodels.jl) provides access to a wide variety of power network formulations for representing AC power flow.

In [27]:
TypeTree(PowerSimulations.PowerModels.AbstractPowerFormulation,scopesep="\n")

PowerModels
AbstractPowerFormulation
├──PowerModels
│  AbstractBFForm
│  ├──PowerModels
│  │  AbstractBFQPForm
│  │  └──PowerModels
│  │     SOCBFForm (0 children)
│  └──PowerModels
│     AbstractBFConicForm
│     └──PowerModels
│        SOCBFConicForm (0 children)
├──PowerModels
│  AbstractWRForm
│  ├──PowerModels
│  │  QCWRForm
│  │  └──PowerModels
│  │     QCWRTriForm (0 children)
│  └──PowerModels
│     SOCWRForm
├──PowerModels
│  AbstractACPForm
│  └──PowerModels
│     StandardACPForm
├──PowerModels
│  AbstractLPACForm
│  └──PowerModels
│     AbstractLPACCForm
├──PowerModels
│  AbstractACTForm
│  └──PowerModels
│     StandardACTForm
├──PowerModels
│  AbstractACRForm
│  └──PowerModels
│     StandardACRForm
├──PowerModels
│  AbstractConicPowerFormulation
│  ├──PowerModels
│  │  AbstractWRMForm
│  │  └──PowerModels
│  │     SDPWRMForm (1 children)
│  └──PowerModels
│     AbstractWRConicForm
│     └──PowerModels
│        SOCWRConicForm (0 children)
└──PowerModels
   AbstractActivePowerFormulation
   ├──CopperPlatePowerModel
   └──PowerModels
      AbstractDCPForm
      ├──PowerModels
      │  AbstractDCPLLForm (1 children)
      └──PowerModels
         DCPlosslessForm (2 children)

The above model is formulated with a `CopperPlatePowerModel` network formulation. In principle, we can adjust the formulation to any concrete subtype of `AbstractPowerFormulation`. 

In [28]:
ed_model = PowerSimulations.EconomicDispatch(sys, PowerSimulations.PowerModels.DCPlosslessForm, 
                                        optimizer = with_optimizer(GLPK.Optimizer));

# Learnings

- Knowledge transfer from Python, R, MATLAB programmers/analysts to Julia
- Best software engineering practices
- Reproducible builds and managing dependencies

# Conclusions

- Free and open source production cost model in Julia
- [PowerSystems.jl](https://github.com/NREL/PowerSystems.jl)
- [PowerSimulations.jl](https://github.com/NREL/PowerSimulations.jl)
- [PowerModels.jl](https://github.com/lanl-ansi/PowerModels.jl)
- [JuMP](https://github.com/JuliaOpt/JuMP.jl)
- [D3TypeTrees](https://github.com/claytonpbarrows/D3TypeTrees.jl)
- Questions?